In [4]:
import pandas as pd
from datetime import datetime

# Load the data from the Excel file
df = pd.read_excel('wesm_exposure.xlsx')

In [5]:
# Get current hour
current_hour = datetime.now().hour

In [8]:
# Function to compare values based on hour
def compare_values(row):
    hour = row['HOUR']
    if hour == current_hour:
        if row['ACTUAL_ENERGY'] > row['TOTAL_BCQ_NOMINATION']:
            return "ACTUAL_ENERGY"
        else:
            return 'TOTAL_BCQ_NOMINATION'
    else:
        return ""

In [9]:
# Apply the comparison function to each row and add a new column 'Higher_Value'
df['Higher_Value'] = df.apply(compare_values, axis=1)

In [12]:
from datetime import datetime

now = datetime.now()
hour = int(now.strftime("%H"))
hour

14

In [10]:
# Print the resulting dataframe
print(df)

    HOUR  ACTUAL_ENERGY  TOTAL_BCQ_NOMINATION  WESM_EXPOSURE   Higher_Value
0      1   61592.267343                 45000   16592.267343               
1      2   60966.452002                 45000   15966.452002               
2      3   60054.578045                 45000   15054.578045               
3      4   57539.090612                 45000   12539.090612               
4      5   57945.359207                 45000   12945.359207               
5      6   60017.748522                 45000   15017.748522               
6      7   60473.073226                 45000   15473.073226               
7      8   70646.604628                 45000   25646.604628               
8      9   83899.691197                 65000   18899.691197               
9     10   90881.797000                 65000   25881.797000               
10    11   95251.090000                 65000   30251.090000               
11    12   95119.704000                 65000   30119.704000               
12    13   9

In [16]:
import openpyxl

wb = openpyxl.load_workbook('wesm_exposure.xlsx')
sheet = wb['Sheet']
# Column containing the hour values
hour_column = 'HOUR'
# Get current hour
now = datetime.now()
current_hour = int(now.strftime("%H"))
# List to store the row values
row_values = []
# Find the index of the hour column (assuming header row)
hour_index = None
for cell in sheet.iter_rows(min_row=1, max_row=1):  # Iterate through header row
  cell_value = cell[0].value
  if cell_value == hour_column:
    hour_index = cell[0].column - 1  # Get column index (minus 1 for 0-based indexing)
    break  # Exit loop after finding the hour column

if hour_index is not None:  # Check if hour column was found
  # Loop through each row
  for row in sheet.iter_rows(min_row=2):  # Start from row 2 (data rows)
    # Get the hour value from the specified column index
    hour_value = row[hour_index].value

    # Check if hour value matches current hour
    if hour_value == current_hour:
      # Extract values from the current row
      for cell in row:
        cell_value = cell.value
        row_values.append(cell_value)
      break  # Exit loop after finding the matching row

# Print the row values (optional)
print(row_values)

[15, 101768.206, 65000, 36768.20600000001]


In [59]:
import plotly.graph_objs as go
import streamlit as st

wb = openpyxl.load_workbook('wesm_exposure.xlsx')
sheet = wb['Sheet']
# Column containing the hour values
hour_column = 'HOUR'
# Get current hour
now = datetime.now()
current_hour = int(now.strftime("%H"))

# Function to compare values based on hour
def compare_values(row):
    # Option 1: Check if `column_index` method exists (modern openpyxl)
    if hasattr(sheet, 'column_index'):  # Check for the attribute
        hour_index = sheet.column_index(hour_column)
    else:  # Fallback for older versions
        for col_idx in range(1, sheet.max_column + 1):  # Iterate through columns
            if sheet.cell(row=1, column=col_idx).value == hour_column:
                hour_index = col_idx
                break  # Exit loop after finding the matching column

    hour = row[hour_index - 1].value
    if hour == current_hour:
        if row[1].value > row[2].value:
            # Load the data from the Excel file
            total_bcq_nomination_wb = openpyxl.load_workbook('total_bcq_nomination.xlsx')
            total_bcq_nomination_sheet = total_bcq_nomination_wb['Sheet']
            wesm_exposure_wb = openpyxl.load_workbook('wesm_exposure.xlsx')
            wesm_exposure_sheet = wesm_exposure_wb['Sheet']
            # Find the row matching the current hour in the second file
            comparison_row = None
            for comparison_row in total_bcq_nomination_sheet.iter_rows():
                comparison_hour = comparison_row[0].value
                if comparison_hour == current_hour:
                    break # Exit loop after the matching hour is found
            comparison_row2 = None
            for comparison_row2 in wesm_exposure_sheet.iter_rows():
                comparison_hour2 = comparison_row2[0].value
                if comparison_hour2 == current_hour:
                    break
            # Check if a matching row was found
            if comparison_row is not None:
                # Extract desired values from the matching row
                scpc_value = comparison_row[1].value
                kspc_value = comparison_row[2].value
                kspc1_value = int(kspc_value/2)
                kspc2_value = int(kspc_value/2)
                edc_value = comparison_row[3].value
            if comparison_row2 is not None:
                wesm_value = comparison_row2[3].value

                # Do something with the extracted values (e.g., print, store in a list)
                print(f"SCPC: {scpc_value}, KSPC1: {kspc1_value}, KSPC2: {kspc2_value}, EDC: {edc_value}, WESM: {wesm_value}")
            else:
                print(f"No matching hour found in comparison data for {current_hour}")


            labels = ['SCPC', 'KSPC1', 'KSPC2', 'EDC', 'WESM']
            sizes = [scpc_value, kspc1_value, kspc2_value, edc_value, wesm_value]

            # Create the donut chart data
            data = [
                go.Pie(
                    labels=labels,
                    values=sizes,
                    hole=0.5,
                    textinfo='label+percent',
                    textposition='inside',
                    marker={'colors': ['lightcoral', 'lightskyblue', 'lightgreen']},
                )
            ]
            # Display the donut chart in Streamlit
            st.plotly_chart(figure=data, use_container_width=True)

        else:
            # Handle the case where ACTUAL_ENERGY is not greater than TOTAL_BCQ_NOMINATION (optional)
            if row[1].value <= row[2].value:
                # Load the data from the Excel file
                total_bcq_nomination_wb = openpyxl.load_workbook('total_bcq_nomination.xlsx')
                total_bcq_nomination_sheet = total_bcq_nomination_wb['Sheet']

                # Find the row matching the current hour in the second file
                comparison_row = None
                for comparison_row in total_bcq_nomination_sheet.iter_rows():
                    comparison_hour = comparison_row[0].value
                    if comparison_hour == current_hour:
                        break # Exit loop after the matching hour is found

                # Check if a matching row was found
                if comparison_row is not None:
                    # Extract desired values from the matching row
                    scpc_value = comparison_row[1].value
                    kspc_value = comparison_row[2].value
                    kspc1_value = int(kspc_value/2)
                    kspc2_value = int(kspc_value/2)
                    edc_value = comparison_row[3].value
                    # Do something with the extracted values (e.g., print, store in a list)
                    print(f"SCPC: {scpc_value}, KSPC1: {kspc1_value}, KSPC2: {kspc2_value}, EDC: {edc_value}")
                else:
                    print(f"No matching hour found in comparison data for {current_hour}")

                labels = ['SCPC', 'KSPC1', 'KSPC2', 'EDC', 'WESM']
                sizes = [scpc_value, kspc1_value, kspc2_value, edc_value, wesm_value]

                # Create the donut chart data
                data = [
                    go.Pie(
                        labels=labels,
                        values=sizes,
                        hole=0.5,
                        textinfo='label+percent',
                        textposition='inside',
                        marker={'colors': ['lightcoral', 'lightskyblue', 'lightgreen']},
                    )
                ]   
                # Display the donut chart in Streamlit
                st.plotly_chart(figure=data, use_container_width=True)

    else:
        # Handle the case where the current hour doesn't match the row's hour (optional)
        pass

# Process rows in the sheet (assuming you want to process all rows)
for row in sheet.iter_rows(min_row=2):  # Start from row 2 (assuming header row)
  compare_values(row)

SCPC: 25000, KSPC1: 10000, KSPC2: 10000, EDC: 20000, WESM: 36396.6955
